In [1]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.chat_models import ChatOpenAI

In [52]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
KEY=os.getenv("OPENAI_API_KEY")

In [31]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5) 

In [9]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [10]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [11]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [32]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [33]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [34]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [35]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [36]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [37]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [51]:
file_path=r"C:\Data\Soft Dev\AI\LLMs\GenerativeAICourse\Projects\mcqgen\data.txt"
file_path

'C:\\Data\\Soft Dev\\AI\\LLMs\\GenerativeAICourse\\Projects\\mcqgen\\data.txt'

In [39]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [40]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [41]:
NUMBER=5 
SUBJECT="Generative AI"
TONE="simple"

In [42]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Artificial intelligence (AI) has the potential to vastly improve the metaverse by automating intelligent decision-making and creating highly customized user experiences. Web3, with its distributed network architecture, provides consumers with enhanced privacy and security when conducting financial transactions online [1], [2], [3]. In addition, the immutable data storage and transfer mechanisms made possible by blockchain technology guarantee data security and integrity. In the age of Web3, generative AI technologies like Chat Generative Pre-trained Transformer (ChatGPT) have the capacity to become productivity tools by addressing problems with digital assets and content productin and filling in essential gaps in Web3’s evolution [4]. Generative AI technologies are expected to accelerate the advent of the Web3 era by offering more reliable and convenient productivity tools for We

In [43]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2656
Prompt Tokens:1802
Completion Tokens:854
Total Cost:0.004411


In [44]:
quiz=response.get("quiz")

In [45]:
quiz=json.loads(quiz)

In [46]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [47]:
quiz_table_data

[{'MCQ': 'What technology is expected to accelerate the advent of the Web3 era?',
  'Choices': 'a: Blockchain technology | b: Generative AI technologies | c: Virtual reality (VR) | d: Augmented reality (AR)',
  'Correct': 'b'},
 {'MCQ': 'Which AI technology is mentioned as a component of the metaverse engine layer?',
  'Choices': 'a: Artificial general intelligence (AGI) | b: Reinforcement learning | c: Chat Generative Pre-trained Transformer (ChatGPT) | d: Natural language processing (NLP)',
  'Correct': 'c'},
 {'MCQ': 'What is one benefit of using ChatGPT in content production and dissemination?',
  'Choices': 'a: Decreased data security | b: Limited creativity | c: Improved efficiency and quality | d: Reduced content variety',
  'Correct': 'c'},
 {'MCQ': 'What is a key challenge in the current metaverse content scale?',
  'Choices': 'a: High cost of building metaverse spaces | b: Lack of AI technology | c: Excessive content variety | d: Low user demand',
  'Correct': 'a'},
 {'MCQ': 

In [48]:
quiz=pd.DataFrame(quiz_table_data)

In [49]:
quiz.to_csv("genai.csv",index=False)

In [1]:
from datetime import datetime

datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'03_15_2024_01_01_31'

In [3]:
import json
with open('C:\Data\Soft Dev\AI\LLMs\GenerativeAICourse\Projects\mcqgen\Response.json', 'r') as file:
    RESPONSE_JSON = json.load(file)

In [15]:
import PyPDF2
def read_file(file):
    if file.endswith(".pdf"):
        try:
            print("yest")
            pdf_reader=PyPDF2.PdfReader(file)
            text=""
            for page in pdf_reader.pages:
                text+=page.extract_text()
            return text
        except Exception as e:
            raise Exception("Error reading the PDF file")
        
    elif file.endswith(".txt"):
        return file.read().decode("utf-8")
    
    else:
        raise Exception(
            "Unsupported file format only pdf and text files are supported"
        )
file_path=r"C:\Data\Soft Dev\AI\python.pdf" 
print(file_path)
#"C:\Data\Soft Dev\AI\python.pdf"
text = read_file(file_path)
print(text)

C:\Data\Soft Dev\AI\python.pdf
yest
1T able of Contents
Introduction
Preface
Introduction to Python
Installing Python
Running Python programs
Python 2 vs Python 3
The basics of working with Python
Data types
Operators
The T ernary Operator
Strings
Booleans
Numbers
Constants
Enums
User Input
Control statements
Lists
Tuples
Dictionaries
Sets
Functions
Objects
Loops2Classes
Modules
The Python Standard Library
The PEP8 Python style guide
Debugging
Variables scope
Accept arguments from the command line
Lambda functions
Recursion
Nested functions
Closures
Decorators
Docstrings
Introspection
Annotations
Exceptions
The with statement
Installing 3rd party packages using pip
List comprehensions
Polymorphism
Operator Overloading
Virtual Environments
Conclusion3Introduction
The Python Handbook follows the 80/20 rule: learn
80% of the topic in 20% of the time.
I find this approach gives a well-rounded overview .
This book does not try to cover  everything under the
sun related to Python. It focuse 

In [3]:
file = r"pikachu or geodude.pdf"
print (file.endswith(".pdf"))

True
